# Step 1 - Clean data

In [18]:
import pandas as pd
import re

## 1.1 Clean data for Property Assessment DB

In [19]:
# Load the data from data/property_assessment.csv
pa_df = pd.read_csv('../data/raw/all_housing.csv')
pa_df.head()

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIPCODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIRE_PLACE,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,,100001000.0,104 A 104,PUTNAM ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,3,A - Average,NaN
1,100002000,,100002000.0,197,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0,NaN,0,A - Average,NaN
2,100003000,,100003000.0,199,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0,NaN,0,A - Average,NaN
3,100004000,,100004000.0,201,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
4,100005000,,100005000.0,203,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN


In [20]:
# Select columns we are interested in
interested_columns = ["OWN_OCC", "LU_DESC", "UNIT_NUM", "ST_NUM", "ST_NAME", "ZIPCODE", "CITY", "RES_UNITS"]
property_assessment = pa_df[interested_columns]
property_assessment

,OWN_OCC,LU_DESC,UNIT_NUM,ST_NUM,ST_NAME,ZIPCODE,CITY,RES_UNITS
0,Y,THREE-FAM DWELLING,NaN,104 A 104,PUTNAM ST,2128,EAST BOSTON,0
1,Y,THREE-FAM DWELLING,NaN,197,LEXINGTON ST,2128,EAST BOSTON,0
2,N,THREE-FAM DWELLING,NaN,199,LEXINGTON ST,2128,EAST BOSTON,0
3,N,THREE-FAM DWELLING,NaN,201,LEXINGTON ST,2128,EAST BOSTON,0
4,Y,TWO-FAM DWELLING,NaN,203,LEXINGTON ST,2128,EAST BOSTON,0
...,...,...,...,...,...,...,...,...
178593,N,CITY OF BOSTON,NaN,,KNOWLES ST,2135,BRIGHTON,0
178594,N,RES LAND (Unusable),NaN,,LAKE ST,2135,BRIGHTON,0
178595,N,THREE-FAM DWELLING,NaN,4,LAKE ST,2135,BRIGHTON,0
178596,N,STRIP CTR STORES,NaN,2193 2201,COMMONWEALTH AV,2135,BRIGHTON,0


In [21]:
# Since ST NUM could be a range of numbers like 100 200 we expand it into a row with one number each
result_rows = []

def expand_st_num(row):
    row = row.to_dict()

    if row["ST_NUM"] is None:
        result_rows.append(row)
        return

    # Split numbers using regex
    numbers = re.findall(r'\d+', row["ST_NUM"])
    st_nums = [int(n) for n in numbers]
    
    if len(st_nums) > 1:
        # Extract all numbers from the string
        first = st_nums[0]
        last = st_nums[-1]

        for i in range(first, last + 1):
            new_row = row.copy()
            new_row["ST_NUM"] = i
            result_rows.append(new_row)
    elif len(st_nums) == 1:
        # Convert to integer
        row["ST_NUM"] = st_nums[0]
        result_rows.append(row)
    else:
        # If street number is not a number, set to None
        row["ST_NUM"] = None
        result_rows.append(row)

property_assessment.apply(expand_st_num, axis=1)
result_df = pd.DataFrame(result_rows)
result_df

,OWN_OCC,LU_DESC,UNIT_NUM,ST_NUM,ST_NAME,ZIPCODE,CITY,RES_UNITS
0,Y,THREE-FAM DWELLING,NaN,104.0,PUTNAM ST,2128,EAST BOSTON,0
1,Y,THREE-FAM DWELLING,NaN,197.0,LEXINGTON ST,2128,EAST BOSTON,0
2,N,THREE-FAM DWELLING,NaN,199.0,LEXINGTON ST,2128,EAST BOSTON,0
3,N,THREE-FAM DWELLING,NaN,201.0,LEXINGTON ST,2128,EAST BOSTON,0
4,Y,TWO-FAM DWELLING,NaN,203.0,LEXINGTON ST,2128,EAST BOSTON,0
...,...,...,...,...,...,...,...,...
275405,N,STRIP CTR STORES,NaN,2198.0,COMMONWEALTH AV,2135,BRIGHTON,0
275406,N,STRIP CTR STORES,NaN,2199.0,COMMONWEALTH AV,2135,BRIGHTON,0
275407,N,STRIP CTR STORES,NaN,2200.0,COMMONWEALTH AV,2135,BRIGHTON,0
275408,N,STRIP CTR STORES,NaN,2201.0,COMMONWEALTH AV,2135,BRIGHTON,0


In [22]:
# Save the result
result_df.to_csv('../data/clean/all_housing.csv', index=False)